##Binary Clasification of Text data. 
The following project can be used for applications where input data is a string of large text data and output is a binary classification of data. An example of the use of the below project could be sentiment classification ofcomments, hate speech detection, etc.

Algorithm used : Random Forest Classifier

Data used for this project: 
@InProceedings{maas-EtAl:2011:ACL-HLT2011,
  author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
  title     = {Learning Word Vectors for Sentiment Analysis},
  booktitle = {Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies},
  month     = {June},
  year      = {2011},
  address   = {Portland, Oregon, USA},
  publisher = {Association for Computational Linguistics},
  pages     = {142--150},
  url       = {http://www.aclweb.org/anthology/P11-1015}
}


Reference: https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34


## Import  necessary packages

In [30]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.metrics import confusion_matrix
import pickle


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Loading And Preprocessing Data

In [ ]:
np.random.seed(500)
Corpus = pd.read_csv("IMDB Dataset.csv",encoding='latin-1',engine='python',error_bad_lines=False)

# 1. : Drop empty rows.
Corpus['review'].dropna(inplace=True)
# 2. : All text is made lower case. Different case words treated as separate words. If this s relevant for the data, you can skip the below step.
Corpus['review'] = [entry.lower() for entry in Corpus['review']]
# 3. : Tokenization(Breaking sentences to words)
Corpus['review']= [word_tokenize(entry) for entry in Corpus['review']]
# 4. : Remove Stop words using WordNetLemmatizer
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['review']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    Corpus.loc[index,'text_final'] = str(Final_words)

Skipping line 34068: unexpected end of data


Divide Text in Train(75%) and Test(25%) sets

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['sentiment'],test_size=0.3)

# Encode Label to 0 and 1

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

# Convert Text to Vectors(Vectorization)

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=10000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

# Train The Model. Here we will use 4 different Algorithms and compare the same

# 3.1 Train - Random Forest Classifier

In [48]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
#Tried with estimators 50, 100, 150. After 100 estimators, the improvement was negligible(<2%)
rf=RandomForestClassifier(n_estimators=100)


# Train the model on training data
rf.fit(Train_X_Tfidf,Train_Y);

## 3.2 Test - Random Forest Classifier

In [49]:
# predict the labels on  test set
predictions_rf = rf.predict(Test_X_Tfidf)
print("RF Accuracy Score -> ",accuracy_score(predictions_rf, Test_Y)*100)
print("RF Precision Score -> ",precision_score(predictions_rf, Test_Y)*100)
print("RF Recall Score -> ",recall_score(predictions_rf, Test_Y)*100)

RF Accuracy Score ->  84.45205479452055
RF Precision Score ->  83.96300669027941
RF Recall Score ->  84.64590359055742


# To predict value for a particular Review or sentence use following code:

In [ ]:
print(Encoder.inverse_transform(Test_Y))

In [ ]:
Encoder.inverse_transform(SVM.predict(Tfidf_vect.transform(["This is a good good day."])))

array(['N'], dtype=object)